# Day 21

### Importing modules and process input

In [14]:
from collections import deque
import time
import functools as ft

start_time = time.time()    # Start the timer
input = [list(line.strip()) for line in open("input").readlines()]
sz_r, sz_c = len(input), len(input[0])

### Part 1

Simple, straightforward BFS by level

In [15]:
next_moves = [(1, 0),(-1, 0),(0, -1),(0, 1)]

start_r, start_c = next((r, c) for r in range(sz_r) for c in range(sz_c) if input[r][c] == 'S')
queue = [(start_r, start_c)]

# BFS for part 1
cycle = 0
while queue:
    sz = len(queue)
    if cycle == 64:
        print(f'Part 1: {sz}')
        break

    visited = set()
    for _ in range(sz):
        r, c = queue.pop(0)

        for move_r, move_c in next_moves:
            new_r, new_c = r + move_r, c + move_c
            if (new_r, new_c) not in visited and new_r >= 0 and new_r < sz_r and new_c >= 0 and new_c < sz_c and input[new_r][new_c] != '#':
                queue.append((new_r, new_c))
                visited.add((new_r, new_c))

    cycle += 1

Part 1: 3733


# Part 2

This is where it's getting fun. Considering that this map is copied to every directions, here are some observations we see:
- If the BFS go out of bound of one copy of the map, it will just end up to the other copy of the map, with location that's in bound of the map size.

→ We should track the info of the map position that we are at, together with location in the bound of the map size, it's easier than tracking only raw location

- If a specific garden plot has been reached first time after k max_steps, then, this is only reached after n max_steps when n - k is an even number (by going back and forth) → n and k must be odd or even together.

Also, there are some other special properties of the map. Firstly, there's no rock in the horizontal and vertical lines that go through the starting point (this one wasn't shown in the input), and no rock on the side lines of the map. Which means that, there are clear and straight paths for us to walk through garden plots. An example plot of this is like what is shown in the image below.

![Day 21 expanding map](img/Day%2021%20expanding%20map.png)

Since I have talked about storing information of both map and location inside the map, it's a cool idea to track the amount of nodes will end up on the same location, but different map (Since the given number is too large). With the above image, a pattern we can see is that, it will gradually enlarge into 8 directions over time. To make it easier to understand, here's another image

![Day 21 tiles expanding](img/Day%2021%20tiles%20expanding.png)

In this one, I consider each map as a tile, and this is an example, when depth is = 5. And you can already notice the coloring I have. My plan of optimization here was to expand it by depth = 1, like this:

![Day 21 tiles expanding starter](img/Day%2021%20tiles%20expanding%20starter.png)

Then, from this, I expand them and calculate using math, based on the reachable distance. And, from each corner of the initial expanded map with depth = 1, I can expand to both top/down and left/right, while for the walls, I only expand to either top/down/left/right only. That's why there was that coloring.

That's the basic idea, but enough talk, let's get into code to understand it better.

This is the first part, to calculate the minimum distance to get to a specific map, on specific location. You can see that, I choose **bound_size** to be 1, because I only expand by one. (It can be expanded more, but that's just time wasting).

In [16]:
bound_size = 1
min_steps_reachable = {}    # Contains the amount of minimum moves needed to reach a specific plot on specific tile

queue = deque([(0, 0, start_r, start_c, 0)])
while queue:
    tile_r, tile_c, plot_r, plot_c, min_steps = queue.popleft()
    
    # Change to the appropriate tile and plot's row and column
    if plot_r < 0:
        tile_r -= 1
        plot_r += sz_r
    if plot_r >= sz_r:
        tile_r += 1
        plot_r -= sz_r
    if plot_c < 0:
        tile_c -= 1
        plot_c += sz_c
    if plot_c >= sz_c:
        tile_c += 1
        plot_c -= sz_c

    if abs(tile_r) > bound_size or abs(tile_c) > bound_size:
        continue
    if not (0 <= plot_r < sz_r and 0 <= plot_c < sz_c and input[plot_r][plot_c] != "#"):
        continue
    if (tile_r, tile_c, plot_r, plot_c) in min_steps_reachable:
        continue
    
    min_steps_reachable[(tile_r, tile_c, plot_r, plot_c)] = min_steps
    for move_r, move_c in next_moves:
        next_r, next_c = plot_r + move_r, plot_c + move_c
        queue.append((tile_r, tile_c, next_r, next_c, min_steps + 1))

Next, I write a function to calculate the amount of duplicated nodes on a specific type of wall (either corner or just wall). Notice how the first parameter is only the amount of minimum steps ? I will explain this later.

In [17]:
max_steps = 26501365    # The amount of given steps in the question

@ft.cache
def count_duplicates(min_steps, pos_type):
    steps_remaining = max_steps - min_steps
    reachable_tiles_cnt = steps_remaining // sz_r   # The amount of reachable maps depth

    start, step = 0, -1
    
    # This is only for further cases of odd or even values of sz_r, 
    if sz_r % 2 == 0:
        if max_steps % 2 == 0 and min_steps % 2 == 0:
            start, step = 1, 1
        elif max_steps % 2 != 0 and min_steps % 2 != 0:
            start, step = 1, 2
    else:
        start, step = (1, 2) if (max_steps % 2 == 0) ^ (min_steps % 2 == 0) else (2, 2)

    if step == -1:
        return 0

    reachable_tiles_cnt -= reachable_tiles_cnt % 2 == 0 if start == 1 else reachable_tiles_cnt % 2

    # The total amount of duplicate moves in the edges
    number_st_size = (reachable_tiles_cnt - start) // step + 1

    # The total amount of duplicate moves in the corner
    total_quad = ((start + reachable_tiles_cnt) * number_st_size // 2) + number_st_size

    return total_quad if pos_type == 'corner' else number_st_size

    # The above optimization was based on this code, to make it O(1)
    # It doesn't start from 0, because the zero itself was added in the code below already
    # for x in range(1, reachable_tiles_cnt + 1):
    #     if (min_steps + sz_r * x) % 2 == max_steps % 2:
    #         occurences += x + 1 if pos_type == 'corner' else 1

And, this is the final part. I travel and add up on all 9 maps (1 original map, and 8 surrounded founding map). Now you can see why I only cache two information of min steps, which is the minimum travel distance needed to be reachable, and type of wall: More optimization. For example, one location, on any corner, are just having similar reachability like any other corner (same with normal walls), because from starting initial point, it's just having same distance, as the travelling is BFS by level. That's why, only type of wall and minimum distance matters.

In [18]:
ans = 0
for r in range(sz_r):
    for c in range(sz_c):
        # Skip unreachable places, like, rocks, and plots that are surrounded by rocks
        # For example:
        # .#.
        # #.#  => This middle plot is blocked entirely
        # .#.
        if (0, 0, r, c) not in min_steps_reachable:
            continue

        for tr in range(-bound_size, bound_size+1):
            for tc in range(-bound_size, bound_size+1):
                min_move = min_steps_reachable[(tr, tc, r, c)]
                
                # The case when the node is reachable in less than the amounf of the goal steps
                if min_move % 2 == max_steps % 2 and min_move <= max_steps:
                    ans += 1
                if abs(tr) == bound_size and abs(tc) == bound_size:
                    ans += count_duplicates(min_move, 'corner')
                elif abs(tr) == bound_size or abs(tc) == bound_size:
                    ans += count_duplicates(min_move, 'wall')

print(f'Part 2: {ans}')
print(f'Elapsed time: {time.time() - start_time} seconds')

Part 2: 617729401414635
Elapsed time: 2.3773083686828613 seconds
